# Course Recommender System

In [2]:
from collections import defaultdict
from itertools import combinations
import math

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from sklearn.model_selection import train_test_split

from coursemate.dataset import Dataset

In [3]:
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', 1000)

In [4]:
%load_ext autoreload
%autoreload 2

## 0. Dataset preparation

In [5]:
dataset = Dataset('../data/Coursera_courses.csv', '../data/Coursera.csv', '../data/Coursera_reviews.csv')
dataset.set_interaction_counts(3, 50)
dataset.show_dataset_details()

Loading Coursera courses...
Loading Coursera reviews...
Segmenting out students with less than 3 or more than 50 reviews...
30719 students, 468 courses, 174219 reviews
Sparsity: 1.21%
Duplicates: 4.54%


In [6]:
dataset.set_train_test_split_by_user()

# For the content-based and CF recommender
train_Xmatrix, train_ymatrix, df_train_X, df_train_y = dataset.get_train_matrix_split(ratio=0.8)
test_Xmatrix, test_ymatrix, df_test_X, df_test_y = dataset.get_test_matrix_split(ratio=0.5)

# For the sequential/KB recommender
train_Xseq, test_Xseq, train_yseq, test_yseq = dataset.get_train_test_sequence_predictions()

Setting the train-test split by user...
Computing the training and test rating matrix...


131100it [00:10, 12361.20it/s]


Computing the test rating matrix split...
Computing the training and test rating matrix...


131100it [00:10, 12723.54it/s]
43119it [00:03, 12946.23it/s]


Computing the training and test list of sequences...


131100it [00:08, 14922.19it/s]
43119it [00:02, 15129.61it/s]


In [7]:
print(f"Training (Content+CF): From {len(df_train_X)} ratings, predict the next {len(df_train_y)} for {df_train_y['reviewers'].nunique()} users")
print(f"Testing (Content+CF): From {len(df_test_X)} ratings, predict the next {len(df_test_y)} for {df_test_y['reviewers'].nunique()} users")

Training (Content+CF): From 119949 ratings, predict the next 11151 for 6631 users
Testing (Content+CF): From 157922 ratings, predict the next 16297 for 7680 users


In [8]:
print(f"Training (Sequential): From {len(train_Xseq)} sequences, predict the next course taken for {df_train_y['reviewers'].nunique()} users")
print(f"Testing (Sequential): From {len(test_Xseq)} sequences, predict the next course taken for {df_test_y['reviewers'].nunique()} users")

Training (Sequential): From 108061 sequences, predict the next course taken for 6631 users
Testing (Sequential): From 35439 sequences, predict the next course taken for 7680 users


## 1. Content-Based Recommendation

In [164]:
from coursemate.model import ContentBasedModel
from sklearn.feature_extraction.text import TfidfVectorizer,  CountVectorizer

model = ContentBasedModel(course_set=dataset.course_set, Vectorizer=TfidfVectorizer, n_features=10000)

unique_reviewers_X = df_test_X['reviewers'].unique()
unique_reviewers_y = df_test_y['reviewers'].unique()
hits = 0
count = 0
for user in unique_reviewers_X:
    if user in unique_reviewers_y:
        user_profile = tuple(df_test_X[df_test_X['reviewers'] == user]['course_id'])
        target = df_test_y[df_test_y['reviewers'] == user]['course_id']

        recommendations_at_5 = model.recommend(user_profile,5)
        recommendations_at_10  = model.recommend(user_profile,10)

        res = len(set(target) & set(recommendations_at_5))

        count += 1
        if res > 0:
            hits += 1
            
    if count == 200:
        break

print("Hitrate", hits / count)

Hitrate 0.26


## 2. Collaborative Filtering Recommendation

## 3. Sequential/Knowledge-Based Recommendation

In [147]:
from coursemate.model import AssociationMiningModel
from coursemate.metrics import calculate_hit_rate_from_next_course_sequences

# cutoff determined empirically, testing the 90th and 99th percentile
model = AssociationMiningModel(cutoff=3, course_set=dataset.course_set)
df_seq = model.get_sequences_data(train_Xseq, train_yseq)
model.fit(df_seq)

108061 ['aws-fundamentals-going-cloud-native', 'python-basics', 'introduction-cybersecurity-cyber-attacks', 'neural-networks-deep-learning', 'deep-neural-network', 'machine-learning-projects', 'introduction-tensorflow', 'convolutional-neural-networks-tensorflow', 'tensorflow-sequences-time-series-and-prediction', 'machine-learning-projects', 'ai-for-everyone', 'introduction-tensorflow', 'classical-sociological-theory', 'feminism-social-justice', 'machine-learning-with-python', 'python-data', 'neural-networks-deep-learning', 'deep-neural-network', 'machine-learning-projects', 'convolutional-neural-networks', 'nlp-sequence-models', 'ai-for-everyone', 'convolutional-neural-networks-tensorflow', 'system-administration-it-infrastructure-services', 'technical-support-fundamentals', 'neural-networks-deep-learning', 'deep-neural-network', 'convolutional-neural-networks', 'nlp-sequence-models', 'what-is-datascience', 'python-operating-system', 'introduction-git-github', 'troubleshooting-debuggi

In [ ]:
calculate_hit_rate_from_next_course_sequences(test_Xseq, test_yseq, model.recommend, 5)

100%|█████████████████████████████████████████████████████████████████████████████| 35439/35439 [06:47<00:00, 86.97it/s]


0.25906487203363526